In [ ]:
#| default_exp server

# Whisper server

Runs a Whisper server that receives and transcribes incoming audio.  

First we import the python modules.  

In [ ]:
#| export
import sys

import zmq
import numpy as np
from fastcore.basics import store_attr

import whisper

Then we import the `asr_teach` modules.

In [ ]:
#| export
from asr_teach.audio import AudioBuffer
from asr_teach.utils import ZMQ_ARGS, PROTOCOL, ADDR, PORT

## Helper functions  

Below we define a couple of helper functions. 

First, the `recv_array` receives a numpy array over a given socket as a two-part message. The first part of the message has the data type and shape of the incoming array. The second part of the array has the raw array bytes. Both messages are used to deserialize the correct numpy array. One final note: the incoming audio is almost always in Short Int format (np.int16). Whisper expects normalized float32 data, so we cast and normalize the int16 data if needed.

In [ ]:
#| export 

def recv_array(socket, flags=0, copy=True, track=False, scale=32768.0):
    '''Receives a numpy array over `socket`.
    
    Assumes a two-part message:
        First message has the array data-type and shape.
        Second message has the raw array bytes. 
        
    Returns the appropriately cast and reshaped numpy array. 
    '''
    # receive the array shape and type
    md = socket.recv_json(flags=flags)
    # receive the array bytes 
    msg = socket.recv(flags=flags, copy=copy, track=track)
    buf = memoryview(msg)
    # convert bytes into a flat audio array
    arr = np.frombuffer(buf, dtype=md['dtype']).reshape(md['shape']).squeeze()
    # make sure that whisper gets normalized float32 data
    if md['dtype'] == 'int16':
        arr = arr.astype(np.float32) / scale
    return arr

The `parse_transcription` function extracts some useful fields from the full output of Whisper's `transcribe` method. It is mainly for convenient re-naming of a few variables. 

In [ ]:
#| export
def parse_transcription(result):
    '''Extracts helpful fields from the output of whisper's `transcribe`.
    
    Information parsed for each segment:
        - The transcribed text.  
        - Start and stop times of the segment in seconds.  
        - Log probability of the transcription.  
        - Probability that the segment had no speech.  
    '''
    parsed = []
    for seg in result['segments']:
        info = {'text': seg['text'],
                'beg': seg['start'],
                'end': seg['end'],
                'logprob': seg['avg_logprob'],
                'no_speech_prob': seg['no_speech_prob']}
        parsed.append(info)
    return parsed

In [ ]:
#| export

class WhisperServer:
    '''Runs a whisper model on audio streaming over a port.
    
    TODO: proper logging
    '''
    def __init__(self,
                 buffer_size: int = 16_000,
                 model_name: str = 'small',
                 threshold: float = 0.,
                 protocol: str = PROTOCOL,
                 addr: str = ADDR,
                 port: int = PORT):
        store_attr()
        
        # create the socket to receive streaming audio
        context = zmq.Context()
        socket = context.socket(zmq.PULL)
        socket.connect(f'{protocol}://{addr}:{port}')
        self.socket = socket
        
        # load the whisper model
        self.model_name = model_name
        self.model = whisper.load_model(model_name)
        
        
    def recv_data(self, zmq_args=ZMQ_ARGS):
        '''Reads in audio array from `self.socket`.
        '''
        arr = recv_array(self.socket, **zmq_args)
        return arr

    
    def transcribe(self):
        '''Runs whisper trascription on the full audio buffer.
        '''
        # get the data over the socket
        print('Waiting for audio...')
        data = self.recv_data()
        print('Audio received.')
        
        # pad it for whisper
        # TODO: insert data into pre-buffered 30-second Whisper window
        audio = whisper.pad_or_trim(data)
        
        # transcribe the audio and parse the results
        print('Parsing audio...')
        result = self.model.transcribe(audio)
        parsed = parse_transcription(result)
        print('Audio parsed')
        
        # TODO: transcribed text shown on simple GUI
        # TODO: simple thresholding to avoid garbage transcriptions
        print('\n'.join(seg['text'] for seg in parsed if seg['logprob'] > -0.9))
        print('\n'.join(str(seg['logprob']) for seg in parsed))

         
    def run(self):
        '''Main server function.
        
        TODO: catch stopping signals (SIGINT, SIGTERM)
        '''
        print('Starting server...')
        while True:
            try:
                self.transcribe()

            except KeyboardInterrupt:
                print('User interrupt, stopping server...')
                break
                
            except Exception as e:
                print(f'Exception: {e}')
                raise
        print('Server stopped')
